In [1]:
import torch
from torch import nn as nn
import torchvision
from torch.utils.data import DataLoader, dataset
from torchvision.transforms import AutoAugment, Normalize, Compose, ToTensor
import torch.nn.functional as F
from torchmetrics import Accuracy
import numpy as np

In [2]:
class accuracy:
    def __init__(self):
        pass
    def update(self, x,y):
        pass
    def reset(self):
        pass

In [3]:
def __normalize__(x):
    return x/255.
normalize = torch.jit.script(__normalize__)

In [4]:
from torchvision.datasets import CIFAR100

In [17]:
data = CIFAR100("./data", train = True, download = True, transform = Compose([AutoAugment(),ToTensor(), normalize]))
test_data = CIFAR100("./data", train = False, download = True, transform = Compose([ToTensor(),  normalize]))

Files already downloaded and verified
Files already downloaded and verified


In [15]:
train_set = DataLoader(data, 128, shuffle = True, num_workers=2)
test_set = DataLoader(test_data, 128, shuffle = False, num_workers=2)

In [19]:
class VGG5(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_1 = nn.Sequential(nn.Conv2d(3, 64, 2,2),                                                                     
                                   nn.BatchNorm2d(64))
        self.seq_2 = nn.Sequential(nn.Conv2d(64, 128, 2,1), nn.GELU(),
                                   nn.MaxPool2d(2,2, ), 
                                   nn.BatchNorm2d(128))
        self.seq_3 = nn.Sequential(nn.Conv2d(128, 256, 2,1, padding = "same"), nn.GELU(),
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(256, ))
        self.seq_4 = nn.Sequential(nn.Conv2d(256, 512, 2,1, padding = "same"), nn.GELU(),
                                    
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(512, ))
        self.seq_5 = nn.Sequential(nn.Conv2d(512, 1024, 2,1, padding = "same"), nn.GELU(),
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(512, ))
        
        self.f = nn.Flatten()
        self.dense = nn.Linear(4608, 100)
    def forward(self, x):
        x = self.seq_1(x)
        x = self.seq_2(x)
        x = self.seq_3(x)
        x = self.seq_4(x)
        x = self.f(x)
        x = self.dense(x)
        return x

In [20]:
model_ = VGG5()
model = torch.nn.DataParallel(model_)
model.to(0)


DataParallel(
  (module): VGG5(
    (seq_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
      (1): GELU(approximate='none')
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
      (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): MaxPool2

In [21]:
model(torch.randn(1,3,32,32)).shape

torch.Size([1, 100])

In [22]:
def loss(x,y):
    return nn.CrossEntropyLoss()(model(x),y)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [ ]:
t = 0
while True:
    L = []
    loss__ = []
    model.train()

    for i, batch in enumerate(train_set):
        x, y = batch[0].cuda(), batch[1].cuda()
    
        loss_ = loss(x,y)
        loss_.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss__.append(loss_.item())

    model.eval()
    with torch.no_grad():
            
        for j, batch in enumerate(test_set):
            cond = 1
            if cond: ### here we do reservoir sampling
                x, y = batch[0].cuda(), batch[1].cuda()
                L.append(sum(torch.argmax(model(x), dim = 1) == y).item())
            else:
                continue
        
    print(t, sum(L)/len(L), sum(loss__)/len(loss__))
    t += 1

0 9.784810126582279 4.456713582548644
1 13.974683544303797 4.1756830355700325
2 15.924050632911392 4.022426725348549
3 18.670886075949365 3.9154669031157825
4 19.835443037974684 3.8155380155119443
5 21.936708860759495 3.722265903297288
6 22.670886075949365 3.6528564938498884
7 23.49367088607595 3.573047728794615
8 25.21518987341772 3.514489137303189
9 26.303797468354432 3.46252060485313
10 26.974683544303797 3.414423695000846
11 28.037974683544302 3.365209956913043
12 28.974683544303797 3.315468720462926
13 29.670886075949365 3.2761672055324933
14 29.89873417721519 3.23571637097527
15 30.70886075949367 3.189812804122105
16 31.240506329113924 3.153160421439754
17 31.31645569620253 3.1202818249802453
18 32.48101265822785 3.0916709405991734
19 32.607594936708864 3.0502752379688154
20 33.45569620253165 3.021946619843583
21 33.9873417721519 2.9943656689675566
22 34.0126582278481 2.9680933568178847
23 34.721518987341774 2.9387630361425296
24 34.55696202531646 2.9151979535436996
25 35.3291139